# 專題（二）：建置 Bert 新聞觀點分類器之資料集

## 專案目標
- 目標：請試著建製 BertForSequenceClassification 看得懂的兩個句子分類問題的資料集  NewsPairDataset
- 資料集 in archive.zip：
    - 包含：train.csv、test.csv、solution.csv
    - 資料來源：https://www.kaggle.com/wsdmcup/wsdm-fake-news-classification
    - 資料中包含兩個新聞標題 title1_zh 和 title2_zh，並且給予這兩篇新聞的相關性，分別可能是：agreed, unrelated, disagreed

## 實作提示
- STEP1：解壓縮 archive.zip，並且讀取 train.csv 和 test.csv 檔案
- STEP2：繼承 torch.utils.data.Dataset 並實作 NewsPairDataset，其中需要用到 bert tokenizer (請參考官方對BertForSequenceClassification的說明)，特別注意兩個句子間必須要有分隔符號 SEP
- STEP3：因為每一個從 NewsPairDataset 來的樣本長度都不一樣，所以需要實作 collate_fn，來zero padding 到同一序列長度
- STEP4：使用 torch.utils.data.DataLoader 來創造 train_loader 和 valid_loader

## 重要知識點：專題結束後你可以學會
- 如何讀取並處理 NLP 資料，產生可以適用 BertForSequenceClassification 兩個句子分類問題的資料集
- 了解 BERT 的 2-Sequence Classification 任務如何進行

In [1]:
# from: https://www.kaggle.com/wsdmcup/wsdm-fake-news-classification
# !unzip archive.zip

In [2]:
!python --version

Python 3.6.5 :: Anaconda, Inc.


In [3]:
# !pip install -q transformers

In [4]:
import os
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split
from tqdm.notebook import tqdm

from transformers import BertTokenizer, BertForSequenceClassification

In [5]:
os.chdir('archive')
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [6]:
df_train = df_train[['title1_zh', 'title2_zh', 'label']].dropna(axis=0).reset_index(drop=True)
df_test = df_test[['id', 'title1_zh', 'title2_zh']].dropna(axis=0).reset_index(drop=True)

In [7]:
ALL_LABELS = ['agreed', 'unrelated', 'disagreed']

In [8]:
MODEL_NAME = 'bert-base-chinese'

In [9]:
df_train.head()

,title1_zh,title2_zh,label
0,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,unrelated
1,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,unrelated
2,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,unrelated
3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,unrelated
4,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,agreed


In [10]:
# 建置數據集
class NewsPairDataset(Dataset):
    def __init__(self, tokenizer, df, max_len=512):
        self.tokenizer = tokenizer
        self.df = df
        self.max_len = max_len

    def __getitem__(self, idx):
        text1 = self.df.loc[idx, 'title1_zh']
        text2 = self.df.loc[idx, 'title2_zh']
        label = self.df.loc[idx, 'label'] if 'label' in self.df.columns else None

        tokens1 = self.tokenizer.tokenize(text1)
        tokens2 = self.tokenizer.tokenize(text2)
        
        all_tokens_len = len(tokens1) + len(tokens2) + 2  # [CLS] [SEP]
        if all_tokens_len > self.max_len:
            bound = (self.max_len - 2) // 2
            tokens1 = tokens1[:bound]
            tokens2 = tokens2[:bound]
       
        input = {}
        
        sentence = ['[CLS]'] + tokens1 + ['[SEP]'] + tokens2
        input['input_ids'] = torch.tensor(self.tokenizer.convert_tokens_to_ids(sentence))

        sep_idx = sentence.index('[SEP]')
        input['token_type_ids'] = torch.tensor(
            [0] * (sep_idx + 1) + [1] * (len(sentence) - sep_idx - 1),
            dtype=torch.long
        )

        input['attention_mask'] = torch.tensor(
            [1] * len(sentence),
            dtype=torch.long
        )


        if label:
            label = torch.tensor(ALL_LABELS.index(label))

        return input, label

    def __len__(self):
        return len(self.df)


def create_mini_batch(samples):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    labels = []
    for s in samples:
        input_ids.append(s[0]['input_ids'].squeeze(0))
        token_type_ids.append(s[0]['token_type_ids'].squeeze(0))
        attention_mask.append(s[0]['attention_mask'].squeeze(0))
        if s[1]:
            labels.append(s[1])

    # zero pad 到同一序列長度
    input_ids = pad_sequence(input_ids, batch_first=True)
    token_type_ids = pad_sequence(token_type_ids, batch_first=True)
    attention_mask = pad_sequence(attention_mask, batch_first=True)
 
    if labels:
        labels = torch.stack(labels)
        return input_ids, token_type_ids, attention_mask, labels
    else:
        return input_ids, token_type_ids, attention_mask

In [11]:
train_batch_size = 32
eval_batch_size = 512

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

dataset = NewsPairDataset(tokenizer, df_train)

train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    collate_fn=create_mini_batch,
    shuffle=True)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    collate_fn=create_mini_batch)

In [12]:
next(iter(train_loader))

(tensor([[ 101, 2669, 2336,  ...,    0,    0,    0],
         [ 101, 1062, 1062,  ...,    0,    0,    0],
         [ 101,  671,  702,  ...,    0,    0,    0],
         ...,
         [ 101, 1920, 1117,  ...,    0,    0,    0],
         [ 101, 2476, 3377,  ...,    0,    0,    0],
         [ 101, 3580, 2094,  ...,    0,    0,    0]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]))